<a href="https://colab.research.google.com/github/charlesmartin14/All-Convnet-Autoencoder-Example/blob/master/Full_Test_Jacobians_Resnets_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for generating Full Jacobian SVD over the test data for ResNet data

Can do the full calculation, and a batched (slow, but handles large memory cases)

Can handle: ResnetXXX with CIFAR10, CIFAR100, SVHN

i.e. CIFAR 10
```
 resnet20_cifar10
 resnet56_cifar10
 resnet110_cifar10
 resnet164bn_cifar10
 resnet272bn_cifar10

full_jacobian()


 resnet542bn_cifar10
 resnet1001_cifar10
 resnet1202_cifar10

full_batched_jacobian()

```






In [0]:
Jdir =  '/content/drive/My Drive/J_resnets_fullsvd'

thismodel = 'resnet20_cifar100'
num_classes= 100
num_data = 10000

batched = False
start_batch = 0
batch_size = 100

device = 'cuda:0'


In [2]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=7a2e5da5d2d6512889ec3cf3f4aec91c6093af30d2b743fad4fa46d23e74ab22
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 17.1 s


In [5]:
!mkdir "$Jdir"
!ls "$Jdir"

mkdir: cannot create directory ‘/content/drive/My Drive/J_resnets_fullsvd’: File exists
data				   Jfull_resnet1202_cifar10__20.csv
Jfull_resnet1001_cifar10__0.csv    Jfull_resnet1202_cifar10__21.csv
Jfull_resnet1001_cifar10__100.csv  Jfull_resnet1202_cifar10__22.csv
Jfull_resnet1001_cifar10__101.csv  Jfull_resnet1202_cifar10__23.csv
Jfull_resnet1001_cifar10__102.csv  Jfull_resnet1202_cifar10__24.csv
Jfull_resnet1001_cifar10__103.csv  Jfull_resnet1202_cifar10__25.csv
Jfull_resnet1001_cifar10__104.csv  Jfull_resnet1202_cifar10__26.csv
Jfull_resnet1001_cifar10__105.csv  Jfull_resnet1202_cifar10__27.csv
Jfull_resnet1001_cifar10__106.csv  Jfull_resnet1202_cifar10__28.csv
Jfull_resnet1001_cifar10__107.csv  Jfull_resnet1202_cifar10__29.csv
Jfull_resnet1001_cifar10__108.csv  Jfull_resnet1202_cifar10__2.csv
Jfull_resnet1001_cifar10__109.csv  Jfull_resnet1202_cifar10__30.csv
Jfull_resnet1001_cifar10__10.csv   Jfull_resnet1202_cifar10__31.csv
Jfull_resnet1001_cifar10__110.csv  Jfull_resnet1202_c

In [6]:
!pip install pytorchcv
!pip install powerlaw
!pip install GPUtil
import GPUtil

     |████████████████████████████████| 440kB 4.6MB/s 
  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24787 sha256=4a16a09fb86f59c8f0807e818576cc2b2ced5d0e4e08ca0f3c0d76c65bf92e53
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=0d1aaa41fcad8e33ca87bfd47f8165fdf1547509bc9119d190d4a0d4cf61f617
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
time: 12.2 s


In [7]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

import gc


/bin/bash: import: command not found
time: 4.66 s


### Run InstallPhysTheory notebook 

In [8]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 847 ms


In [9]:
def jacobian_batched_full(modelname, model, data_loader, batch_size, start_batch = 0, num_classes=10, device='cuda:0', data_dim=3*32*32, formJJ=False, save=True): 
  '''Compute a full of J^{T}J Jacobian in batch mode.  '''

  model.eval() 
  model = model.to(device)

  left_data_loader = data_loader 
  right_data_loader = copy.deepcopy(data_loader)

  JJMat = None 
  for left_batch, data in enumerate(left_data_loader): 
    if left_batch < start_batch:
      continue
    print(left_batch) 

    inputs, _ = data 
    inputs = inputs.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)
    J = jac.batch_diagJ(inputs, outputs)
    J = J.reshape(batch_size,num_classes*data_dim)

    X = J.detach().cpu().numpy()

    del data, inputs, outputs, J
    gc.collect()
    torch.cuda.empty_cache()
    GPUtil.showUtilization()

    JJBlock = None
    for right_batch, data in enumerate(right_data_loader):
      print(left_batch, right_batch)
      inputs, _ = data 
      inputs = inputs.to(device)
      inputs.requires_grad=True
      outputs = model(inputs)
    
      Jt = jac.batch_diagJ(inputs, outputs)
      Jt = Jt.reshape(batch_size,num_classes*data_dim)

      Xt = Jt.detach().cpu().numpy().transpose()

      del data, inputs, outputs, Jt
      gc.collect()
      torch.cuda.empty_cache()

      block = np.dot(X,Xt)
      JJBlock = np.hstack([JJBlock, block]) if JJBlock is not None else block

    # end of right_batch loop
    
    if save:
      filename = "{}/Jfull_{}__{}.csv".format(Jdir, modelname, left_batch)
      np.savetxt(filename, JJBlock, delimiter='\t')
      print("Saved file ",filename)

    if formJJ:
      JJMat = np.vstack([JJMat, JJBlock]) if JJMat is not None else JJBlock

      print("JJMat shape ", left_batch, JJMat.shape, filename) 

  # end of left_batch loop

  return JJMat


time: 54.1 ms


In [10]:
def jacobian_full(model, data_loader, batch_size, num_classes=10, device='cuda:0'): 
  '''compute a full of J^{T}J Jacobian directly.  '''

  model.eval()
  model = model.to(device)

  J_right = jac.construct_diagJ(model, test_loader, batch_size=batch_size, device=device) 
  J_left = J_right.transpose(1,0)
  JJ = torch.mm(J_right, J_left)
  JJMat = JJ.detach().cpu().numpy()

  del J_right, J_left, JJ
  gc.collect()
  torch.cuda.empty_cache()

  return JJMat

time: 6.8 ms


### ResNet 

In [11]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

time: 96.5 ms


### Compute JJ, in batches or all at once


In [12]:
def get_datasets(thismodel, batch_size):
  print("getting dataset for ", thismodel)
  if thismodel.endswith('cifar100'):
    print("loading cifar100")
    train_dataset = datasets.CIFAR100(
      root='data', 
      train=True, 
      transform=transforms.ToTensor(),
      download=True)
    test_dataset = datasets.CIFAR100(
      root='data', 
      train=False, 
      transform=transforms.ToTensor(),
      download=True) 
 
    train_loader = DataLoader(
      dataset=train_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)
    test_loader = DataLoader(
      dataset=test_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)

  else:
    print("loading cifar10")
    train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

      
  return train_dataset, test_dataset, train_loader, test_loader

time: 9.69 ms


In [13]:
print(thismodel)
train_dataset, test_dataset, train_loader, test_loader = get_datasets(thismodel, batch_size)#utils.get_data(batch_size=batch_size)
#train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

print(test_dataset)


resnet20_cifar100
getting dataset for  resnet20_cifar100
loading cifar100


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()
time: 7.59 s


In [14]:
model = ptcv_get_model(thismodel, pretrained=True)  
if batched: 
  jacobian_batched_full(thismodel, model, test_loader, batch_size=batch_size, start_batch=start_batch)
else:
  JJMat = jacobian_full(model, test_loader, batch_size=batch_size)

RuntimeError: ignored

time: 2min 3s


In [0]:
if batched:
  print("Reading batched JJBlocks for ", thismodel)
  JJMat = None
  import os, glob
  os.chdir(Jdir)
  for filename in glob.glob("Jfull_{}*".format(thismodel)):
    print(filename)
    JJBlock = np.loadtxt(filename)
    JJMat = np.vstack([JJMat, JJBlock]) if JJMat is not None else JJBlock


In [0]:
import powerlaw
norm = 10*3*32*32
evals = np.linalg.eigvals(JJMat)
evals  = (1.0/ norm)*evals

fit = powerlaw.Fit(evals)
alpha = fit.alpha
D = fit.D
max_eval = np.max(evals)

plt.hist(evals, bins=100)
plt.title("{}  alpha = {:0.2f} max  = {:0.2f} ".format(thismodel, alpha, max_eval))
plt.show()

print(alpha, D, max_eval)
